# Individual Assemblies
Because everything clumped together is not good

## Assembly
Flye seems to still not be working properly, use raven

In [ ]:
#========== RAVEN ASSEMBLER ==========
# Raven 1.8.1
conda deactivate
conda activate raven_racon
mkdir 02_assembly_raven

for barcode in $(find /data/WINONA/202208_exxon_fastq/ -maxdepth 1 -name "sample*" -type f); do
    file_name=$(basename $barcode)
    sample_name=${file_name%.*}
    
    echo $sample_name
    raven -p 3 -t 50 $barcode > 02_assembly_raven/${sample_name}_raven_assembled.fasta
done


In [ ]:
#========== MEDAKA NORMAL ==========
cp /data/WINONA/202208_exxon_sepsamples/02_assembly_raven ./
mkdir 03_correction_medaka

for barcode in $(find /data/WINONA/202208_exxon_fastq/ -maxdepth 1 -name "sample*" -type f); do
    file_name=$(basename $barcode)
    sample_name=${file_name%.*}
    
    echo $sample_name

    medaka_consensus \
    -i $barcode \
    -d ./02_assembly_raven/${sample_name}_raven_assembled.fasta \
    -o 03_correction_medaka/$sample_name \
    -m r941_min_sup_g507 \
    -t 32

    #Check depth
    samtools depth -a 03_correction_medaka/$sample_name/calls_to_draft.bam > 03_correction_medaka/$sample_name/depth.txt
    cat 03_correction_medaka/$sample_name/depth.txt | awk -F'\t' '{ sum += $3 } END { print sum / NR }' > 03_correction_medaka/$sample_name/mean_coverage.txt
done



In [ ]:
# MV TO WINONA
sudo mv ./* /data3/WINONA3/202208_exxon_septreatment/
cd /data3/WINONA3/202208_exxon_septreatment/
sudo chown -R winona:winona ./*


In [ ]:
# MAP SAMPLES TO CONTIGS
mkdir 04_mapped_samples
for barcode in $(find /data/WINONA/202208_exxon_fastq/ -maxdepth 1 -name "sample*" -type f); do
    file_name=$(basename $barcode)
    sample_name=${file_name%.*}
    mapped=./04_mapped_samples/${sample_name}_mapped.sam
    bam_raw=./04_mapped_samples/${sample_name}_raw.bam
    bam_sorted=./04_mapped_samples/${sample_name}_sorted.bam
    bam_indexed=./04_mapped_samples/${sample_name}_sorted.bam.bai
    contigs=03_correction_medaka_treated/consensus.fasta

    echo $sample_name
    echo "using"
    echo $contigssam
    
    minimap2 -ax map-ont $contigs $barcode -t 64 > $mapped
    samtools view -o $bam_raw -F 4 -@ 64 -bS $mapped 
    samtools sort -o $bam_sorted -@ 64 $bam_raw 
    samtools index -b -@ 64 $bam_sorted $bam_indexed 
done

